# 0.0 IMPORTS

In [126]:
import pandas   as pd
import numpy    as np
import seaborn  as sns
import inflection
import warnings
import re
import umap.umap_ as umap
import matplotlib.cm as cm
import datetime

from matplotlib    import pyplot     as plt
from plotly        import express    as px
from scipy.cluster import hierarchy  as hc


from sklearn             import cluster       as c
from sklearn             import metrics       as mt
from sklearn             import preprocessing as pp
from sklearn             import decomposition as dd
from sklearn.manifold    import TSNE
from sklearn             import ensemble      as en
from sklearn.mixture     import GaussianMixture
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

warnings.filterwarnings('ignore')

## 0.2 Loading Data

In [127]:
path = 'C:/Users/edils/repos/customer_fidelity/data/'
df_raw = pd.read_csv(path + 'ecommerce.csv', encoding='windows-1252')
df_raw = df_raw.drop('Unnamed: 8', axis=1)

# 1.0 DATA DESCRIPTION

In [128]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [129]:
df1.columns = df1.columns.map(lambda x: inflection.underscore(x))

## 1.2 Data Dimensions

In [130]:
df1.shape

## 1.3 Data Types

In [131]:
df1.dtypes

## 1.4 Check NA

In [132]:
df1.isna().sum()

## 1.5 Replace NA

In [133]:
df_missing = df1.loc[pd.isnull(df1['customer_id']),:]

#create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup),1)

#merge
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

#coalesce - combina o que tem NaN em uma coluna com o que não tem em outra
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

#drop extra columns
df1 = df1.drop(['customer_id_x','customer_id_y'], axis=1)

## 1.6 Change Types

In [134]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])

## 1.7 Descriptive Statistics

In [135]:
numerical_attributes = df1.select_dtypes(include=['float64', 'int64', 'datetime64[ns]'])
categorical_atributtes = df1.select_dtypes(exclude=['float64', 'int64', 'datetime64[ns]'])

## 1.7.1 Numerical Analysis

In [136]:
disp = numerical_attributes.agg(['mean','median','std','min','max',lambda x: max(x) - min(x), 'skew', 'kurtosis']).T.rename(columns={'<lambda>':'range'})

In [137]:
disp

Podemos ver que existes dados grandes e negativos em 'quantity'e também produtos igual a 0.0 em unit_price

## 1.7.2 Categorical Atributtes

In [138]:
categorical_atributtes

# 2.0 VARIABLE FILTERING

In [139]:
df2 = df1.copy()

## 2.1 Numerical Attributes

In [140]:
#unit_price
df2 = df2.loc[df2['unit_price'] >= 0.040,:]

#stock code
df2 = df2.loc[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK']),:]

#description
df2 = df2.drop('description', axis=1)

#country
df2 = df2.loc[~df2['country'].isin(['Unspecified','European Community']),:]

#bad users

df2 = df2.loc[~df2['customer_id'].isin([16446]),:]
df2 = df2.loc[~df2['customer_id'].isin([12346]),:]



In [141]:
#divide df in purchases and returns
df_purchase = df2.loc[df2['quantity'] >=0, :]
df_returns = df2.loc[df2['quantity'] < 0,:]

# 3.0 FEATURE ENGINEERING

In [142]:
df3 = df2.copy()

In [143]:
#create df_ref
df_ref = pd.DataFrame()

## 3.1 Monetary 

In [144]:
#Monetary - O quanto cada cliente gera de $
aux_monetary = pd.DataFrame()
aux_monetary['valor_bruto'] = df_purchase['quantity'] * df_purchase['unit_price']
aux_monetary['customer_id'] = df_purchase['customer_id']
df_ref = aux_monetary.loc[:,['valor_bruto', 'customer_id']].groupby('customer_id').sum().reset_index().rename(columns={'valor_bruto':'monetary'})
df_ref.isna().sum()

## 3.2 Unique Products

In [145]:
uniq_prod = (df_purchase.loc[:,['customer_id','stock_code']].groupby('customer_id')
                       .nunique()
                       .reset_index()
                       .rename(columns={'stock_code':'unique_prods'}))
df_ref = pd.merge(df_ref, uniq_prod, on='customer_id', how='left')
df_ref.isna().sum()

## 3.3 Quantity of Products

In [146]:
uniq_prod = (df_purchase.loc[:,['customer_id','stock_code']].groupby('customer_id')
                       .count()
                       .reset_index()
                       .rename(columns={'stock_code':'qt_prods'}))
df_ref = pd.merge(df_ref, uniq_prod, on='customer_id', how='left')
df_ref.isna().sum()

## 3.4 Average Basket Size

In [147]:
#qtd de produtos diferentes dentro de uma compora
basket_size = (df_purchase.loc[:,['customer_id','stock_code','invoice_no']]
                         .groupby(['customer_id'])
                         .agg(
                         number_buys = ('invoice_no', 'nunique'),
                         number_prods = ('stock_code', 'count'))
                         .reset_index())
basket_size['avg_basket_size'] = basket_size['number_prods'] / basket_size['number_buys']
basket_size = basket_size.drop(['number_prods','number_buys'], axis=1)
df_ref = pd.merge(df_ref, basket_size, on='customer_id', how='left')
df_ref.isna().sum()

## 3.5 Recency

In [148]:
#Recency
aux_recency = df_purchase.loc[:,['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
date_max = df_purchase['invoice_date'].max()
aux_recency['recency'] = (date_max - aux_recency['invoice_date']).dt.days
df_ref = pd.merge(df_ref, aux_recency, on='customer_id', how='left')
df_ref = df_ref.drop(['invoice_date'],axis=1)
df_ref.isna().sum()

## 3.6 Relationship Duration

In [149]:
aux_min = df_purchase.loc[:,['customer_id', 'invoice_date']].groupby('customer_id').min().reset_index()
aux_max = df_purchase.loc[:,['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
aux_relationship = aux_max.copy()
aux_relationship['invoice_date'] = aux_max['invoice_date'].sub(aux_min['invoice_date']).dt.days
aux_relationship = aux_relationship.rename(columns={'invoice_date':'relationship_duration'})
df_ref = pd.merge(df_ref, aux_relationship, on='customer_id', how='left')
df_ref.isna().sum()

## 3.7 Purchase Count

In [150]:
aux_invoice = df_purchase.loc[:,['invoice_no','customer_id']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'invoice_no':'purchase_count'})
df_ref = pd.merge(df_ref, aux_invoice, on='customer_id', how='left')
df_ref.isna().sum()

## 3.8 Returns Count

In [151]:
qtd_returns = df_returns.loc[:,['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'returns_count'})
qtd_returns['returns_count'] = qtd_returns['returns_count']*-1

#merge
df_ref = pd.merge(df_ref, qtd_returns, on='customer_id', how='left')

#Os NaNs gerados são pessoas que não devolveram
df_ref.loc[df_ref['returns_count'].isna(),'returns_count'] = 0
df_ref.isna().sum()

## 3.9 Monetary Return

In [152]:
aux_monetary_return = pd.DataFrame()
aux_monetary_return['valor_bruto'] = df_returns['quantity'] * df_returns['unit_price']
aux_monetary_return['customer_id'] = df_returns['customer_id']
aux_monetary_return = (aux_monetary_return.loc[:,['customer_id', 'valor_bruto']]
                                        .groupby('customer_id').sum()).reset_index()
aux_monetary_return = aux_monetary_return.rename(columns={'valor_bruto':'monetary_returns'})

#merge values
df_ref = pd.merge(df_ref, aux_monetary_return, on='customer_id', how='left')
df_ref.loc[df_ref['monetary_returns'].isna(),'monetary_returns'] = 0
df_ref.isna().sum()

## 3.10 Average Unit Price

In [153]:
aux_unit_price = df_purchase.loc[:,['customer_id','unit_price']].groupby('customer_id').mean().reset_index().rename(columns={'unit_price':'avg_unit_price'})
df_ref = pd.merge(df_ref, aux_unit_price, on='customer_id', how='left')
df_ref.isna().sum()

## 3.11 Return Rate

In [154]:
df_ref['return_rate'] = df_ref['returns_count'] / df_ref['purchase_count']
df_ref.isna().sum()

## 3.12 Average Purchase Interval

In [155]:
df_ref['avg_purchase_interval'] = df_ref.apply(lambda x: (x['purchase_count'] / x['relationship_duration']) if x['relationship_duration'] != 0 else x['relationship_duration'], axis=1)
df_ref.isna().sum()

## 3.13 Frequency

In [156]:
df_ref['frequency'] = df_ref.apply(lambda x: (x['purchase_count'] / x['relationship_duration']) if x['relationship_duration'] != 0 else 0, axis=1)

## 3.15 Average Order Value

In [157]:
df_ref['avg_order_value'] = df_ref.apply(lambda x: (x['monetary'] / x['purchase_count']) if x['purchase_count'] != 0 else x['monetary'], axis=1)

## 3.16 Check Final Result

In [158]:
df_ref.head()

# 4.0 EDA (EXPLORATORY DATA ANALYSIS

In [159]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

In [160]:
df4.shape

## 4.1 Unilateral Analysis

## 4.3 Embedding

In [161]:
#selected dataset

df43 = df4.drop('customer_id', axis=1).copy()

In [162]:
#Transform
mms = pp.MinMaxScaler()
transf_list = df43.columns

for i in transf_list:
    df43[i] = mms.fit_transform(df43[i].values.reshape(-1,1))
    
X = df43.copy()    

## 4.3.4 Tree-Based Embedding

In [163]:
X = df43.drop('monetary', axis=1).copy()
y = df43['monetary'].copy()

In [164]:
#model definition
rf = en.RandomForestRegressor(random_state=42, n_estimators=100, criterion='squared_error')

rf.fit(X, y)

In [165]:
#create leafs
df_leafs = pd.DataFrame(rf.apply(X))

In [166]:
df_leafs.head()

In [167]:
reducer = umap.UMAP(random_state=42, n_neighbors=120, min_dist=0.015)
embedding = reducer.fit_transform(df_leafs)

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x='embedding_x', 
                y='embedding_y',
                data=df_tree)

# 7.0 HYPERPARAMETER FINE TUNNING(Find Clusters)

In [168]:
#X = df6.drop('customer_id', axis=1)
X = df_tree.copy()

In [169]:
X.head()

# 8.0 MODEL TRAINING

## 8.3 HCluster - FAVORITO

In [170]:
k=5
#model definition & training
hc_model = hc.linkage(X, 'ward')

#model predict
labels_hc = hc.fcluster(hc_model, k, criterion='maxclust')

In [171]:
df9 = X.copy()
df9['clusters'] = labels_hc
sns.scatterplot(data=df9, x="embedding_x", y="embedding_y", hue="clusters", palette="deep")

In [172]:
print(f'SS Score: {mt.silhouette_score(X, labels_hc):.3f}')

## 9.5 Cluster Profile

In [173]:
df95 = df4.copy()
df95['clusters'] = labels_hc

In [174]:
df95.head()

**Lembrete**: NÃO podemos explicar porque o customer 1 e 2 estão no mesmo clusters com esses dados, porque eles foram classificados em um espaço de embedding criado pelo algoritmo em alta dimensão. Se for especificamente você precisar explicar o porquê o os customers se parecem será necessário encontrar os clusters no espaço original ao invés do embedding.

In [175]:
#Per Monetary
df_monetary = df95.loc[:,['clusters','monetary']].groupby('clusters').mean().reset_index()

#Per Frequency
df_frequency = df95.loc[:,['clusters','frequency']].groupby('clusters').mean().reset_index()

#Per Recency
df_recency = df95.loc[:,['clusters','recency']].groupby('clusters').mean().reset_index()

#Per Pencentual
df_client_perc = df95.loc[:,['customer_id','clusters']].groupby('clusters').count().reset_index()
df_client_perc['customer_id'] = df_client_perc['customer_id'].apply(lambda x: 100*(x)/(df_client_perc['customer_id'].sum()))
df_client_perc = df_client_perc.rename(columns={'customer_id':'percent'})

#Per qtd_products
df_prods = df95.loc[:,['clusters','qt_prods']].groupby('clusters').mean().reset_index()

#Per Returns
df_returns = df95.loc[:,['clusters','returns_count']].groupby('clusters').mean().reset_index()

#Per Relationship days
df_duration = df95.loc[:,['clusters','relationship_duration']].groupby('clusters').mean().reset_index()

#Per Customer
df_customer = df95.loc[:,['clusters','customer_id']].groupby('clusters').count().reset_index()

df_resume = (pd.merge(df_monetary, df_frequency, on='clusters', how='left')
             .merge(df_recency, on='clusters', how='left')
             .merge(df_client_perc, on='clusters', how='left')
             .merge(df_prods, on='clusters', how='left')
             .merge(df_returns, on='clusters', how='left')
             .merge(df_duration, on='clusters', how='left')
             .merge(df_customer, on='clusters', how='left')
             )


In [176]:
df_resume

# 11.0 DEPLOY TO PRODUCTION

In [177]:
cols_selected = ['customer_id', 'monetary', 'recency', 'qt_prods', 'frequency', 'returns_count', 'relationship_duration', 'clusters']
df_deploy = df95[cols_selected].copy()
#change types
df_deploy['customer_id'] = df_deploy['customer_id'].astype(int)
df_deploy['returns_count'] = df_deploy['returns_count'].astype(int)
df_deploy['clusters'] = df_deploy['clusters'].astype(int)
df_deploy['relationship_duration'] = df_deploy['relationship_duration'].astype(int)
df_deploy['qt_prods'] = df_deploy['qt_prods'].astype(int)
df_deploy['recency'] = df_deploy['recency'].astype(int)




#save last training
df_deploy['last_training'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [178]:
df_deploy.head()

In [179]:
df_deploy.dtypes

In [180]:
import sqlite3
from sqlalchemy import create_engine, text

In [181]:
# #create table
# query_create_table_insiders = """
#     CREATE TABLE insiders (
#         customer_id INTEGER,
#         monetary    REAL,
#         recency     INTEGER,
#         qtd_prods   INTEGER,
#         frequency   REAL,
#         qtd_returns INTEGER,
#         clusters    INTEGER   
#     )
#     """  
 
# conn = sqlite3.connect('insiders_db.sqlite') 
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# #insert data - append
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df_deploy.to_sql('insiders', con=conn, if_exists='append', index=False)
# conn.close()

# #insert data - replace
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df_deploy.to_sql('insiders', con=conn, if_exists='replace', index=False)
# conn.close()

# #select data

In [182]:
# #create database
# conn = sqlite3.connect('insiders_db.sqlite') 
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

In [183]:
# #database coneection
# conn = sqlite3.connect('insiders_db.sqlite')

# #drop table
# query_drop_insiders = """
#     DROP TABLE insiders
# """

# #create table
# query_create_table_insiders = """
#     CREATE TABLE insiders (
#         customer_id           INTEGER,
#         monetary              REAL,
#         recency               INTEGER,
#         qt_prods              INTEGER,
#         frequency             REAL,
#         returns_count         INTEGER,
#         clusters              INTEGER,
#         relationship_duration INTEGER,
#         last_training         DATE
#     )
#     """
# conn.execute(query_drop_insiders)
# conn.commit()

# conn.execute(query_create_table_insiders)
# conn.commit()

In [184]:
# #insert data - append
# conn = create_engine('sqlite:///insiders_db.sqlite')
# df_deploy.to_sql('insiders', con=conn, if_exists='append', index=False)

In [185]:
#consulting database
engine = create_engine('sqlite:///insiders_db.sqlite')
query = """
    SELECT * FROM insiders
"""

df = pd.read_sql_query(sql=text(query), con=engine.connect())

In [186]:
df